Importação das bibliotecas necessárias:

In [ ]:
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

Web Scraping:


Utilizando as libs Requests e BeautifulSoup para obter acesso ao site desejado, e pegar o código HTML onde os dados que eu quero extrair estão inseridos.

In [ ]:
url = 'https://www.nba.com/news/history-mvp-award-winners'

site = requests.get(url)

soup = BeautifulSoup(site.text, 'html.parser')

texto = soup.find_all('p')
    
conteudo = texto[4:72] # passando o slice para retirar coisas indesejadas que possam atrapalhar nas próximas etapas.

Extração dos dados desejados:


Utilizando de RegEx para encontrar os padrões da temporada onde os jogadores foram MVP's, seus nomes, e nome dos times em que eles jogavam nessa respectiva temporada.

In [ ]:
lista_mvps = []
temporadas = []  # listas onde serão armazenados os dados.
times = []


padrao_nome_jogador = re.compile(r"[A-Z].*,") 
padrao_temporada = re.compile(r"[0-9]{4}-[0-9]{2}")  # criação dos padrões de RegEx.
padrao_times = re.compile(r"(?<=, ).*([A-Z]+|\d+)[a-z]+.*") 


for linha in conteudo:  # extração do conteudo desejado.
    nome_jogador = re.search(padrao_nome_jogador, linha.text)
    lista_mvps.append(nome_jogador.group()[:-1])

    temporada = re.search(padrao_temporada, linha.text)
    temporadas.append(temporada.group())

    time = re.search(padrao_times, linha.text)
        
    if '(Voting)' in time.group():
        time = time.group().replace(' (Voting)', '')
        times.append(time)
    else:
        times.append(time.group())


Transformando em um DataFrame:

In [ ]:
dados = {
    "Temporada": temporadas,
    "Jogador": lista_mvps,
    "Time": times
}

dados_df = pd.DataFrame(dados)
dados_df.head()


Apresentando graficamente:

In [ ]:
cores = ['green', 'yellow', 'blue', 'red', 'green']

top5_times = dados_df['Time'].value_counts().head()

grafico_times = top5_times.plot.bar(width=0.7, color=cores)
grafico_times.set_title("Times com mais jogadores eleitos MVP")
grafico_times.set_ylabel("Quantidade")

plt.grid(grafico_times, axis='y')
plt.show(grafico_times)


In [ ]:
top5_mvps = dados_df['Jogador'].value_counts().head()

grafico_jogadores = top5_mvps.plot.bar(width=0.7, color='r')
grafico_jogadores.set_title("Jogadores com mais prêmios de MVP")
grafico_jogadores.set_ylabel("Quantidade")

plt.show(grafico_jogadores)
